# Proyecto integrador

Miguel Ángel Pérez López A01750145

Ariadna Huesca Coronado A01749161

Lenguaje: GO

https://golang.org/ref/spec#Floating-point_literals

http://www.regular-expressions.info/unicode.html#prop

In [2]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

Variable

In [111]:
;;; Regular expression:
(def GO-variable #"(?xi)
                    [\p{L}_][\p{L}\p{N}_]*") 
(is (re-matches GO-variable "a"))
(is (re-matches GO-variable "_x9"))
(is (re-matches GO-variable "ThisVariableIsExported"))
(is (re-matches GO-variable "αβ"))
(is (not (re-matches GO-variable ".")))

true

STRING

In [119]:
(def GO-string #"(?xi)
                    `[^`]*`|
                    \"([^\\]|
                        \\ 
                        ([abfnrtv\\\"`]|
                          U[\dA-F]{4}|
                          [0-7]{3}|
                          x[\dA-F]{2}))* 
                    \"") 
(is (re-matches GO-string "`abc`"))
(is (re-matches GO-string "`\n \n`"))
(is (re-matches GO-string "\"\\\"\""))
(is (re-matches GO-string "\"Hello, world!\\n\""))
(is (re-matches GO-string "\"日本語\""))
(is (re-matches GO-string "\"\\u65e5本\\U00008a9e\""))
(is (re-matches GO-string "\"\\xff\\u00FF\""))
(is (not (re-matches GO-string "\"\\0\"")))
(is (not (re-matches GO-string "\"\\xa\"")));este era con U
(is (not (re-matches GO-string "\"\\UabcZ\"")));este era con U
(is (not (re-matches GO-string "\"\\\"")));este era con U

true

In [4]:
(def GO-string #"(?xi)
                    import|var|true|false") 
(is (re-matches GO-string "import"))
(is (re-matches GO-string "var"))
(is (re-matches GO-string "true"))
(is (re-matches GO-string "false"))
(is (not (re-matches GO-string " ")))
(is (not (re-matches GO-string "importar")))
(is (not (re-matches GO-string "_var")))

true

## Tareas

1. Entero 1
2. FLotante 1
3. Comentario 1
4. Variable 1
5. String
6. Operadores (Asignación) = += *= Mmike
7. Lógico (> < ==) Mike
8. Aritmético(+ - *) mike
9. Constantes (True, False, var) Ari
10. Paréntesis y llaves Ari

1. fn2: exportar archivo html
2. fn1: (inicio intermedio final) : string 
3. intermedio: ("<span class=" " resultado(float,var) ">"  contenido "</span>"
4. contenido: (< (menor que)   ➔   &lt; > (mayor que)   ➔   &gt; & (ampersand)   ➔   &amp;)

In [ ]:
#| 0[bB][01]+(?:_[01]+)* | [0]+(?:_\d+)*\d+ | [1-9]+\d*(?:_\d+)*    
#| 0[xX][_]*([0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*|[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)(?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*

In [8]:
(def regex #"(?xi)
  (0[oO][_]*[0-7]+(?:_[0-7]+)* | 0[xX][0-9a-fA-F]+(?:_[0-9a-fA-F]+)*)   # Grupo 1: Enteros 
  | (\d*(?:_\d)*\.\d*(?:[eE][+-]*\d+(?:_\d)*)*)                         # Grupo 2: Flotantes  
  | ( // .* )                                                         # Grupo 3: Comentario
  | ( [\p{L}_][\p{L}\p{N}_]* )                                        # Grupo 4: Variable
  | (`[^`]*`|\"(?:[^\\]|\\(?:[abfnrtv\\\"`]|U[\dA-F]{4}|[0-7]{3}|x[\dA-F]{2}))*\")# Grupo 5: String
  | ( [*+]= )                                                         # Grupo 6: Operador de Asignacion
  | ( [-] )                                                           # Grupo 7: Operador Lógico
  | ( [*] )                                                           # Grupo 8: Operador Aritmético
  | (import|var|true|false)                                         # Grupo 9: Constantes
  | ( [(){}] )                                                        # Grupo 10: Puntuación
  | ( \s )                                                            # Grupo 11: Espacios
  | ( . )                                                             # Grupo 12: Carácter inválido  
  ")

#'user/regex

In [11]:
(defn tokenize-file
  [file-name]
  (->> (re-seq regex (slurp file-name))
       (remove (fn [match] (match 11)))
       (map (fn [match]
              (let [token (match 0)]
                (cond
                  (match 1) [token "Entero"]
                  (match 2) [token "Flotante"]
                  (match 3) [token "Comentario"]
                  (match 4) [token "Variable"]
                  (match 5) [token "String"]
                  (match 6) [token "Asignacion"]
                  (match 7) [token "Logico"]
                  (match 8) [token "Aritmetico"]
                  (match 9) [token "Constante"]
                  (match 10) [token "Puntuacion"]
                  (match 11) [token "Invalido"]
                  (match 12) [token "Invalido"]
                  ))))))


#'user/tokenize-file

In [13]:
(tokenize-file "ejemplo.txt")

(["var" "Variable"] ["myvariable1" "Variable"] ["=" "Invalido"] ["2" "Invalido"] ["0" "Invalido"] ["var" "Variable"] ["myvariable2" "Variable"] ["=" "Invalido"] ["\"GeeksforGeeks\"" "String"] ["var" "Variable"] ["myvariable3" "Variable"] ["=" "Invalido"] ["34.80" "Flotante"])

In [14]:
(defn table-lines
  [file-name]
  (->> (tokenize-file "ejemplo.txt")
      (map (fn [t] (format "%-30s%s" (t 0) (t 1))))))

#'user/table-lines

In [15]:
(defn line
  [character size]
  (clojure.string/join (repeat size character)))

#'user/line

In [16]:
(defn table-string 
  [file-name]
  (clojure.string/join \newline
    (concat [(line \= 50)]
            [(format "%-30s%s" "Token" "Tipo")]
            [(line \= 50)]
            (table-lines file-name)
            [(line \= 50)])))

#'user/table-string

In [17]:
(defn print-token-table
  [file-name]
  (println (table-string file-name)))

#'user/print-token-table

In [18]:
(print-token-table "ejemplo.txt")

Token                         Tipo
var                           Variable
myvariable1                   Variable
=                             Invalido
2                             Invalido
0                             Invalido
var                           Variable
myvariable2                   Variable
=                             Invalido
"GeeksforGeeks"               String
var                           Variable
myvariable3                   Variable
=                             Invalido
34.80                         Flotante


nil

In [19]:
(spit "salida.html" (table-string "ejemplo.txt"))

nil

In [34]:
(def mike #"(?xi)
  0[oO][_]*[0-7]+(?:_[0-7]+)*                 #octal
| 0[xX][0-9a-fA-F]+(?:_[0-9a-fA-F]+)*         #hexadecimal
| 0[bB][01]+(?:_[01]+)*                       #binario
| (?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*)[eE][+-]??(?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*) #flotantes con E sin punto
| [0]+\d*(?:_\d+)*\d+                         #enteros empezando con 0
| [1-9]+\d*(?:_\d+)*                          #enteros que no empiezan con 0
| \d*(?:_\d)*\.\d*(?:[eE][+-]??\d+(?:_\d)*)*        #flotantes con enteros
| 0[xX][_]*([0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*|[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)(?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)* #hexadecimales flotantes
| >>={0,1}|<<={0,1} #aritmetico
| [=><!]=| [><] #comparacion
| [-+|^*&/%]??= #asignacion
| |[|]{2}|&{2}|! #logico
| [-+*/%&|^]|(&^) #aritmetico
| (?:// .* )                                  #Comentario")

#'user/mike

In [36]:
;enteros
(is (re-matches mike "4_4543_35435_2"))
(is (re-matches mike "342_3242_2342"))
(is (re-matches mike "7_7"))
(is (re-matches mike "00_232_232"))
(is (re-matches mike "0_232_232"))
(is (re-matches mike "8699874238403009"))
(is (re-matches mike "43216400"))
(is (re-matches mike "00089970"))
(is (re-matches mike "08989"))
(is (re-matches mike "7678575"))
(is (re-matches mike "0o6565467"))
(is (re-matches mike "0x435F564A"))
(is (re-matches mike "0X000_3_f565"))
(is (re-matches mike "0x47464456f_32f_3_3"))
(is (re-matches mike "0b10100_01_1"))
(is (re-matches mike "0B00_1_111"))
(is (re-matches mike "0o_6_3432_2_34"))
(is (re-matches mike "0O4_43_2432"))
(is (re-matches mike "0_54654654"))
(is (re-matches mike "//Hello"))
(is (re-matches mike "0xBad_Face"))
;flotantes enteros
(is (re-matches mike "0."))
(is (re-matches mike "72.40"))
(is (re-matches mike "7_2.40"))
(is (re-matches mike "072.40"))
(is (re-matches mike "2.71828"))
(is (re-matches mike "1.e+0"))
(is (re-matches mike "1.e+0_4_4_4"))
(is (re-matches mike "6.67428e-11"))
;checar este caso
(is (re-matches mike ".25"))
(is (re-matches mike ".12345E+5"))
(is (re-matches mike "1_5."))
(is (re-matches mike "0.15e+0_2"))
;flotantes con e sin punto
(is (re-matches mike "1E6"))
(is (re-matches mike "0E6"))
(is (re-matches mike "434_43e423"))
(is (re-matches mike "00565_443_4e4_4"))
(is (not (re-matches mike "4534545e_e")))
;flotantes hexadecimales
(is (re-matches mike "0x2.p10"))
(is (re-matches mike "0x2_3.p10_12_2222_2"))
(is (re-matches mike "0X1.p1"))
(is (re-matches mike "0x_4.p43"))
(is (re-matches mike "0x_4p45"))
(is (re-matches mike "0x1p-2"))
(is (re-matches mike "0x3_3._32p3"))
(is (re-matches mike "0x1.Fp+0"))
(is (re-matches mike "0x.8p-0"))
(is (re-matches mike "0X_1FFFP-16"))
(is (re-matches mike "0x15e-2"))
(is (re-matches mike "0x.1p1"))
;este caso es erróneo
(is (not (re-matches mike "0x.p1")))
;operadores
(is (re-matches mike "+="))
(is (re-matches mike "-="))
(is (re-matches mike "|="))
(is (re-matches mike "=="))
(is (re-matches mike "^="))
(is (re-matches mike "<"))
(is (re-matches mike ">"))
(is (re-matches mike ">>"))
(is (re-matches mike "<<"))
(is (re-matches mike "<<="))
(is (re-matches mike ">>="))
(is (not (re-matches mike ">>>=")))
(is (not (re-matches mike "<<<=")))
(is (re-matches mike "||"))
(is (re-matches mike "&&"))
(is (re-matches mike "!"))

["!" nil nil]

## Version final hexadecimal

0[xX][_]*([0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*|[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)(?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*

In [ ]:
;0[oO][_]*[0-7]+(_[0-7]+)* #octal
;0[xX][0-9a-fA-F]+(_[0-9a-fA-F]+)* #hexadecimal
;0[bB][01]+(_[01]+)* #binario

;[0]+(_\d+)*\d+ #enteros empezando con 0
;[1-9]+\d*(_\d+)* #enteros que no empiezan con 0

;\d*(_\d)*\.\d*([eE][+-]*\d+(_\d)*)* #flotantes con enteros

;#flotantes hexadecimales como 0x2_3.p10_12_2222_2 sin tener numeros despues del .
;0[xX][_]*[0-9a-fA-F]+(_[0-9a-fA-F])*\.p[0-9a-fA-F]+(_[0-9a-fA-F]+)* 
;final casi bueno 0[xX][_]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[pP][+-]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*
;caso que acepta 1.p 0[xX][_]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[pP][+-]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*
;caso que acepta .1p 0[xX][_]*[0-9a-fA-F]*(_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*[pP][+-]*[0-9a-fA-F]+(_[0-9a-fA-F]+)*

## Asignación

In [29]:
(def mike2 #"(?xi)
 >>={0,1}|<<={0,1} #aritmetico
| [=><!]=| [><] #comparacion
| [-+|^*&/%]??= #asignacion
| |[|]{2}|&{2}|! #logico
|[-+*/%&|^]|(&^) #aritmetico")

#'user/mike2

In [33]:
(println
 (is (re-matches mike2 "+="))
(is (re-matches mike2 "-="))
(is (re-matches mike2 "|="))
 (is (re-matches mike2 "=="))
(is (re-matches mike2 "^="))
(is (re-matches mike2 "<"))
(is (re-matches mike2 ">"))
(is (re-matches mike2 ">>"))
(is (re-matches mike2 "<<"))
(is (re-matches mike2 "<<="))
(is (re-matches mike2 ">>="))
(is (not (re-matches mike2 ">>>=")))
(is (not (re-matches mike2 "<<<=")))
(is (re-matches mike2 "||"))
(is (re-matches mike2 "&&"))
(is (re-matches mike2 "!")))

[+= nil] [-= nil] [|= nil] [== nil] [^= nil] [< nil] [> nil] [>> nil] [<< nil] [<<= nil] [>>= nil] true true [|| nil] [&& nil] [! nil]


nil